## Cluster tweets

In [3]:
from twitterscraper import query_tweets
import codecs, json
import re
import operator
import copy
import nltk
import numpy as np
import pandas as pd
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, SpaceTokenizer, PunktSentenceTokenizer
from collections import Counter
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import copy
from tqdm import tqdm

INFO: {'User-Agent': 'Opera/9.80 (X11; Linux i686; Ubuntu/14.10) Presto/2.12.388 Version/12.16'}


In [3]:
with codecs.open('collection.json', 'r', 'utf-8') as f:
    tw = json.load(f, encoding = 'utf-8')

In [4]:
len(tw)

122564

In [4]:
fr = dict()
for i in tqdm(range(len(tw))):
    if tw[i]['text'] not in fr:
        fr[tw[i]['text']] = 1
    else:
        fr[tw[i]['text']] += 1

100%|██████████| 122564/122564 [00:00<00:00, 1304860.51it/s]


In [10]:
len(fr)

120480

In [11]:
twits = sorted(fr.items(), key=lambda item: (-item[1], item[0]))

In [14]:
for i in range(8):
    print(twits[i])

('This is a , and we need the  to ACT LIKE IT.    ask the future president about their  plan! ', 135)
('I’ve just signed the open letter to stand in solidarity with  survivors and force the Australian government  to recognize & address the . Will you do the same?   ', 95)
('.@sussanley  : Urgent Action Plan Needed to  from Extinction    - Sign the Petition!  via ', 88)
("In the wake of the worst bushfire crisis in Australia's history, NSW & Victoria are STILL issuing permits that allow wildlife like kangaroos, wombats, and possums to be shot and killed. This must end NOW.   SPEAK OUT: … v/  ", 63)
('Pray for Australia  ', 47)
('Hi    and  — cities need to take charge on climate action! Will you lead the way by declaring a  and implementing an emissions reform plan in your city?', 37)
('The planet is in crisis. Tell congress to declare a  now: ', 36)
('Pray for Australia ', 31)


Здесь наоборот много спама - в обработанных сообщениях. Так что лучше смотреть в необработанных, а потом обрабатывать

## BERT

In [2]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [4]:
import torch
from transformers import *

In [5]:
sent_2020_1 = pd.read_csv('sent_2020_1.csv')
sent_2020_2 = pd.read_csv('sent_2020_2.csv')
sent_2020_3 = pd.read_csv('sent_2020_3.csv')
sent_2020_4 = pd.read_csv('sent_2020_4.csv')
sent_2020_5 = pd.read_csv('sent_2020_5.csv')
sent_2020_6 = pd.read_csv('sent_2020_6.csv')
sent_2020_7 = pd.read_csv('sent_2020_7.csv')
sent_2020_8 = pd.read_csv('sent_2020_8.csv')
sent_2020_9 = pd.read_csv('sent_2020_9.csv')
sent_2020_10 = pd.read_csv('sent_2020_10.csv')

In [6]:
#sent_2020_1_0 = sent_2020_1.query('cluster_3 == 0')

In [7]:
punct_list = list(punctuation)
punct_list.append('``')
for j in ['…', '\'', '’', '', '...', '/…', '\'\'', '“', '”']:
    punct_list.append(j)

In [8]:
for_bert = []
for i in range(len(sent_2020_1)):
    filtered_word = ''
    the_text = sent_2020_1['text'].iloc[i]
    tok_list = word_tokenize(the_text)
    for word in tok_list:
        if word.lower() not in punct_list: 
            my_word = word.lower()
            my_word_num = re.sub(r" .*?[0-9]*.*? ", " ", my_word)
            my_word_num = re.sub(r"[.\'/]", "", my_word_num)
            filtered_word += my_word_num
            filtered_word += ' '
    for_bert.append(filtered_word)

In [9]:
for_bert

['water a commidity that is bought and sold to the highest bidder ',
 'and i guess if you get rid of the bush there will be no bushfires ',
 'happy new year and greetings from a country on fire ',
 'would like to know the camera you use ',
 'photography is a great ho by and interest ',
 'this is true compassion true heartbreak true leadership ',
 'deuteronomy 4:24 king james versionen ',
 'for the lord thy god is a consuming fire even a jealous god ',
 'heading off to work this morning and the smoke outside here has an interesting flavour hint of caramel latte ',
 'we need to take asap to halt co2 emissions at levels our civilization can handle or we ll look back on these days as a mild preview of what s to come ',
 'i ve seen nothing from him on tv or here ',
 'does anyone know if the minister for home affairs has commented on recently ',
 'a baby receiving a bravery medal for his dad ',
 'look scott morrison his governments inability to handle this crisis is in no way indicative of t

In [10]:
from tqdm import tqdm
MODELS = (BertModel,       BertTokenizer,       'bert-base-uncased')

#надо передать без пунктуации и в нижнем регистре, остальное обрабатывать не надо
pretrained_weights = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(pretrained_weights)

model = BertModel.from_pretrained(pretrained_weights,
                                        output_hidden_states=True,
                                        output_attentions=True).cuda()
model = model.eval()

bert_embedd = []

with torch.no_grad():
    for i in tqdm(range(len(for_bert))):
        input_ids = torch.tensor([tokenizer.encode(for_bert[i])]).cuda()
        all_hidden_states, all_attentions = model(input_ids)[-2:]
        sent_vec = all_hidden_states[-1].squeeze(0).mean(axis = 0).cpu().numpy()

        bert_embedd.append(sent_vec)

100%|██████████| 9483/9483 [01:20<00:00, 118.09it/s]


In [12]:
print(len(bert_embedd))
print(len(for_bert))
#print(len(sent_2020_1_0))

9483
9483


In [13]:
sent_2020_1['berts'] = bert_embedd

In [14]:
from tqdm import tqdm

In [15]:
def bert_vecs(table):
    sent_2020 = table
    for_bert = []
    for i in range(len(sent_2020)):
        filtered_word = ''
        the_text = sent_2020['text'].iloc[i]
        tok_list = word_tokenize(the_text)
        for word in tok_list:
            if word.lower() not in punct_list: 
                my_word = word.lower()
                my_word_num = re.sub(r" .*?[0-9]*.*? ", " ", my_word)
                my_word_num = re.sub(r"[.\'/]", "", my_word_num)
                filtered_word += my_word_num
                filtered_word += ' '
        for_bert.append(filtered_word)

    MODELS = (BertModel,       BertTokenizer,       'bert-base-uncased')

    #надо передать без пунктуации и в нижнем регистре, остальное обрабатывать не надо
    pretrained_weights = 'bert-base-uncased'
    tokenizer = BertTokenizer.from_pretrained(pretrained_weights)

    model = BertModel.from_pretrained(pretrained_weights,
                                            output_hidden_states=True,
                                            output_attentions=True).cuda()
    model = model.eval()

    bert_embedd = []

    with torch.no_grad():
        for i in tqdm(range(len(for_bert))):
            input_ids = torch.tensor([tokenizer.encode(for_bert[i])]).cuda()
            all_hidden_states, all_attentions = model(input_ids)[-2:]
            sent_vec = all_hidden_states[-1].squeeze(0).mean(axis = 0).cpu().numpy()

            bert_embedd.append(sent_vec)
        
    sent_2020['berts'] = bert_embedd
    return sent_2020

In [16]:
#list(sent_2020_1_b['berts'])[0]

In [17]:
sent_2020_1_b = bert_vecs(sent_2020_1)
sent_2020_2_b = bert_vecs(sent_2020_2)
sent_2020_3_b = bert_vecs(sent_2020_3)
sent_2020_4_b = bert_vecs(sent_2020_4)
sent_2020_5_b = bert_vecs(sent_2020_5)
sent_2020_6_b = bert_vecs(sent_2020_6)
sent_2020_7_b = bert_vecs(sent_2020_7)
sent_2020_8_b = bert_vecs(sent_2020_8)
sent_2020_9_b = bert_vecs(sent_2020_9)
sent_2020_10_b = bert_vecs(sent_2020_10)

100%|██████████| 9482/9482 [01:24<00:00, 111.63it/s]


In [37]:
# sent_2020_1_b.to_csv('sent_2020_1_b.csv')
# sent_2020_2_b.to_csv('sent_2020_2_b.csv')
# sent_2020_3_b.to_csv('sent_2020_3_b.csv')
# sent_2020_4_b.to_csv('sent_2020_4_b.csv')
# sent_2020_5_b.to_csv('sent_2020_5_b.csv')
# sent_2020_6_b.to_csv('sent_2020_6_b.csv')
# sent_2020_7_b.to_csv('sent_2020_7_b.csv')
# sent_2020_8_b.to_csv('sent_2020_8_b.csv')
# sent_2020_9_b.to_csv('sent_2020_9_b.csv')
# sent_2020_10_b.to_csv('sent_2020_10_b.csv')

In [39]:
# sent_2020_1_b = pd.read_csv('sent_2020_1_b.csv')
# sent_2020_2_b = pd.read_csv('sent_2020_2_b.csv')
# sent_2020_3_b = pd.read_csv('sent_2020_3_b.csv')
# sent_2020_4_b = pd.read_csv('sent_2020_4_b.csv')
# sent_2020_5_b = pd.read_csv('sent_2020_5_b.csv')
# sent_2020_6_b = pd.read_csv('sent_2020_6_b.csv')
# sent_2020_7_b = pd.read_csv('sent_2020_7_b.csv')
# sent_2020_8_b = pd.read_csv('sent_2020_8_b.csv')
# sent_2020_9_b = pd.read_csv('sent_2020_9_b.csv')
# sent_2020_10_b = pd.read_csv('sent_2020_10_b.csv')

In [88]:
from scipy.spatial import distance
from tqdm import tqdm

In [92]:
def cosine(table):
    sent_2020 = table
    vecs = sent_2020['berts']
    kmeans = KMeans(n_clusters=3).fit(list(vecs))
    
    cluster_of_3 = []
    for i in range(len(sent_2020)):
        cluster_of_3.append(kmeans.predict([list(vecs)[i]])[0])
    
    sent_2020['cluster_3'] = np.array(cluster_of_3)
    
    cosine_dist = []
    for i in tqdm(range(len(sent_2020))):
        center = cluster_of_3[i]
        dist = distance.cosine(kmeans.cluster_centers_[center], list(vecs)[i])
        cosine_dist.append(dist)
    
    sent_2020['cosine_dist'] = cosine_dist
    return sent_2020

In [99]:
sent_2020_1_bist = cosine(sent_2020_1_b)
sent_2020_2_bist = cosine(sent_2020_2_b)
sent_2020_3_bist = cosine(sent_2020_3_b)
sent_2020_4_bist = cosine(sent_2020_4_b)
sent_2020_5_bist = cosine(sent_2020_5_b)
sent_2020_6_bist = cosine(sent_2020_6_b)
sent_2020_7_bist = cosine(sent_2020_7_b)
sent_2020_8_bist = cosine(sent_2020_8_b)
sent_2020_9_bist = cosine(sent_2020_9_b)
sent_2020_10_bist = cosine(sent_2020_10_b)

100%|██████████| 9482/9482 [00:04<00:00, 2112.41it/s]


In [100]:
#list(sent_2020_1_bist.query('cluster_3 == 1').sort_values(by = 'cosine_dist')['text'])

In [2]:
sent_2020_1_bist.to_csv('sent_2020_1_bist.csv')
sent_2020_2_bist.to_csv('sent_2020_2_bist.csv')
sent_2020_3_bist.to_csv('sent_2020_3_bist.csv')
sent_2020_4_bist.to_csv('sent_2020_4_bist.csv')
sent_2020_5_bist.to_csv('sent_2020_5_bist.csv')
sent_2020_6_bist.to_csv('sent_2020_6_bist.csv')
sent_2020_7_bist.to_csv('sent_2020_7_bist.csv')
sent_2020_8_bist.to_csv('sent_2020_8_bist.csv')
sent_2020_9_bist.to_csv('sent_2020_9_bist.csv')
sent_2020_10_bist.to_csv('sent_2020_10_bist.csv')

NameError: name 'sent_2020_1_bist' is not defined

In [4]:
sent_2020_1_bist = pd.read_csv('sent_2020_1_bist.csv')
sent_2020_2_bist = pd.read_csv('sent_2020_2_bist.csv')
sent_2020_3_bist = pd.read_csv('sent_2020_3_bist.csv')
sent_2020_4_bist = pd.read_csv('sent_2020_4_bist.csv')
sent_2020_5_bist = pd.read_csv('sent_2020_5_bist.csv')
sent_2020_6_bist = pd.read_csv('sent_2020_6_bist.csv')
sent_2020_7_bist = pd.read_csv('sent_2020_7_bist.csv')
sent_2020_8_bist = pd.read_csv('sent_2020_8_bist.csv')
sent_2020_9_bist = pd.read_csv('sent_2020_9_bist.csv')
sent_2020_10_bist = pd.read_csv('sent_2020_10_bist.csv')

In [38]:
print(list(sent_2020_1_bist.query('cluster_3 == 0').sort_values(by = 'cosine_dist')['text'])[1])
print(list(sent_2020_1_bist.query('cluster_3 == 0').sort_values(by = 'cosine_dist')['timestamp'])[1])

Just a reminder that about 60% of Australia is on fire, half a billion animals have died 15 people have died I currently live where the black circle is Not trying to annoy the TL but this major issue is only starting to get some coverage internationally  
2020-01-03T22:07:22


In [39]:
print(list(sent_2020_1_bist.query('cluster_3 == 1').sort_values(by = 'cosine_dist')['text'])[0])
print(list(sent_2020_1_bist.query('cluster_3 == 1').sort_values(by = 'cosine_dist')['timestamp'])[0])

 Promoting his government’s superior action on an emergency crisis NOW I HAVE SEEN IT ALL    :/…
2020-01-04T06:11:37


In [58]:
print(list(sent_2020_1_bist.query('cluster_3 == 2').sort_values(by = 'cosine_dist')['text'])[3])
print(list(sent_2020_1_bist.query('cluster_3 == 2').sort_values(by = 'cosine_dist')['timestamp'])[3])

200+ homes have been destroyed 500 MILLION animals burnt alive and the fires are STILL going after 3 months …
2020-01-04T01:10:43


In [59]:
#list(sent_2020_2_bist.query('cluster_3 == 0').sort_values(by = 'cosine_dist')['text'])

In [60]:
#list(sent_2020_2_bist.query('cluster_3 == 1').sort_values(by = 'cosine_dist')['text'])

In [62]:
#list(sent_2020_2_bist.query('cluster_3 == 2').sort_values(by = 'cosine_dist')['text'])

In [63]:
#list(sent_2020_3_bist.query('cluster_3 == 0').sort_values(by = 'cosine_dist')['text'])

In [40]:
print(list(sent_2020_3_bist.query('cluster_3 == 1').sort_values(by = 'cosine_dist')['text'])[0])
print(list(sent_2020_3_bist.query('cluster_3 == 1').sort_values(by = 'cosine_dist')['timestamp'])[0])

These pictures from Australia are just heartbreaking Almost 15 million hectares burnt & 480 million animals have been killed by bushfires in Australia so far
2020-01-05T13:40:35


In [64]:
#list(sent_2020_3_bist.query('cluster_3 == 2').sort_values(by = 'cosine_dist')['text'])

In [65]:
#list(sent_2020_4_bist.query('cluster_3 == 0').sort_values(by = 'cosine_dist')['text'])

In [66]:
print(list(sent_2020_4_bist.query('cluster_3 == 1').sort_values(by = 'cosine_dist')['text'])[3])
print(list(sent_2020_4_bist.query('cluster_3 == 1').sort_values(by = 'cosine_dist')['timestamp'])[3])

ongoing fires in Australia have killed 500 million animals,have burned 15 million acres and 30% of the koala population has perished during one of the worst natural disasters in Australia,absolutely heartbreaking times for Australia
2020-01-06T14:00:32


In [41]:
#list(sent_2020_4_bist.query('cluster_3 == 2').sort_values(by = 'cosine_dist')['text'])

In [42]:
print(list(sent_2020_5_bist.query('cluster_3 == 0').sort_values(by = 'cosine_dist')['text'])[0])
print(list(sent_2020_5_bist.query('cluster_3 == 0').sort_values(by = 'cosine_dist')['timestamp'])[0])

Now we hear the truth about the Victimized by arsonists and nonsensical global warming tree clearing rulesCan the media stop lying about the cause, this kind of idiotic policy is going to get more people homeless and killed 
2020-01-07T21:57:05


In [67]:
#list(sent_2020_5_bist.query('cluster_3 == 1').sort_values(by = 'cosine_dist')['text'])

In [43]:
print(list(sent_2020_5_bist.query('cluster_3 == 2').sort_values(by = 'cosine_dist')['text'])[0])
print(list(sent_2020_5_bist.query('cluster_3 == 2').sort_values(by = 'cosine_dist')['timestamp'])[0])

 it is reported over 500 million animals have died in the fires killing 10,000 Indian camels is a step the Australians are taking so the countries drinking water doesn't run out
2020-01-07T18:01:05


In [44]:
#list(sent_2020_6_bist.query('cluster_3 == 0').sort_values(by = 'cosine_dist')['text'])

In [45]:
print(list(sent_2020_6_bist.query('cluster_3 == 1').sort_values(by = 'cosine_dist')['text'])[0])
print(list(sent_2020_6_bist.query('cluster_3 == 1').sort_values(by = 'cosine_dist')['timestamp'])[0])

   Thousands of volunteer crafters are making bat wraps,bird nests,joey pouches,etc for very few animal survivors of the fires that are ravaging Australia
2020-01-10T02:46:12


In [46]:
#list(sent_2020_6_bist.query('cluster_3 == 2').sort_values(by = 'cosine_dist')['text'])

In [47]:
#list(sent_2020_7_bist.query('cluster_3 == 0').sort_values(by = 'cosine_dist')['text'])

In [68]:
print(list(sent_2020_7_bist.query('cluster_3 == 1').sort_values(by = 'cosine_dist')['text'])[2])
print(list(sent_2020_7_bist.query('cluster_3 == 1').sort_values(by = 'cosine_dist')['timestamp'])[2])

:: ' World Rallies To Demand Climate Action As Australia Burns ' |  |  The Ferocity & Tragedy Of Australia’s Bushfires, Which Have Claimed 26 Lives, Razed More Than 2,000 Homes, Have Shocked The Nation
2020-01-11T15:38:11


In [48]:
#list(sent_2020_7_bist.query('cluster_3 == 2').sort_values(by = 'cosine_dist')['text'])

In [49]:
#list(sent_2020_8_bist.query('cluster_3 == 0').sort_values(by = 'cosine_dist')['text'])

In [50]:
#list(sent_2020_8_bist.query('cluster_3 == 1').sort_values(by = 'cosine_dist')['text'])

In [51]:
#list(sent_2020_8_bist.query('cluster_3 == 2').sort_values(by = 'cosine_dist')['text'])

In [52]:
#list(sent_2020_9_bist.query('cluster_3 == 0').sort_values(by = 'cosine_dist')['text'])

In [53]:
#list(sent_2020_9_bist.query('cluster_3 == 1').sort_values(by = 'cosine_dist')['text'])

In [54]:
print(list(sent_2020_9_bist.query('cluster_3 == 2').sort_values(by = 'cosine_dist')['text'])[0])
print(list(sent_2020_9_bist.query('cluster_3 == 2').sort_values(by = 'cosine_dist')['timestamp'])[0])

An interesting piece on the impact of Australia's fires, which have already destroyed nearly 1,900 homes
2020-01-24T11:58:01


In [55]:
#list(sent_2020_10_bist.query('cluster_3 == 0').sort_values(by = 'cosine_dist')['text'])

In [56]:
print(list(sent_2020_10_bist.query('cluster_3 == 1').sort_values(by = 'cosine_dist')['text'])[1])
print(list(sent_2020_10_bist.query('cluster_3 == 1').sort_values(by = 'cosine_dist')['timestamp'])[1])

Yes, about 35% of Australian bushfires are started by arsonists but the extensive spread and intensity this season is escalated due to drought ravaged landscapes, erratic weather and a dire lack of preparedness
2020-01-31T04:52:58


In [57]:
#list(sent_2020_10_bist.query('cluster_3 == 2').sort_values(by = 'cosine_dist')['text'])